In [1]:
# Import libaries
import pandas as pd
import requests
from bs4 import BeautifulSoup

### Step 1: Create a soup object from the home page

In [2]:
url = 'http://www.chubbygrub.com/'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')

### Step 2: Scrape the home page soup for every restaurant

Note: Your best bet is to create a list of dictionaries, one for each restaurant. Each dictionary contains the restaurant's name and slug. The result of your scrape should look something like this:

```python
restaurants = [
    {'name': 'A&W Restaurants', 'slug': 'aw-restaurants'}, 
    {'name': "Applebee's", 'slug': 'applebees'},
    ...
]
```

In [3]:
soup.find('a', {'class': 'btn btn-lg btn-primary'}).attrs['href']

'/restaurants/aw-restaurants'

In [4]:
restaurants = []
for restaurant in soup.find_all('a', 'btn btn-lg btn-primary'):
    temp = {}
    temp['name'] = restaurant.text
    temp['slug'] = restaurant.attrs['href'].replace('/restaurants/', '')
    restaurants.append(temp)
restaurants [:4]

[{'name': 'A&W Restaurants', 'slug': 'aw-restaurants'},
 {'name': "Applebee's", 'slug': 'applebees'},
 {'name': "Arby's", 'slug': 'arbys'},
 {'name': 'Atlanta Bread Company', 'slug': 'atlanta-bread-company'}]

### Step 3: Using the slug, scrape each restaurant's page and create a single list of food dictionaries.

Your list of foods should look something like this:
```python
foods = [
    {
        'calories': '0',
        'carbs': '0',
        'category': 'Drinks',
        'fat': '0',
        'name': 'A&W® Diet Root Beer',
        'restaurant': 'A&W Restaurants'
    },
    {
        'calories': '0',
        'carbs': '0',
        'category': 'Drinks',
        'fat': '0',
        'name': 'A&W® Diet Root Beer',
        'restaurant': 'A&W Restaurants'
    },
    ...
]
```

**Note**: Remove extra white space from each category

In [5]:
urls = []
for i in range(len(restaurants)):
    url = 'http://chubbygrub.com/restaurants/' + restaurants[i]['slug']
    urls.append(url)
urls[:3]

['http://chubbygrub.com/restaurants/aw-restaurants',
 'http://chubbygrub.com/restaurants/applebees',
 'http://chubbygrub.com/restaurants/arbys']

In [6]:
restaurants[0]

{'name': 'A&W Restaurants', 'slug': 'aw-restaurants'}

In [7]:
# for i in range(1):
#     url = 'http://chubbygrub.com/restaurants/' + restaurants[i]['slug']
#     response = requests.get(url)
#     soup = BeautifulSoup(response.text, 'lxml')
    
#     temp = {}
#     for item in soup.find_all('tr', {'itemtype': "http://schema.org/NutritionInformation"}):
#         temp['calories'] = soup.find('td', {'itemprop': "calories"}).text
#         temp['carbs'] = soup.find('td', {'itemprop': 'carbohydrateContent'}).text
#         temp['category'] = soup.find('td', {'class': 'hidden-xs category-column'}).text.replace('\n', '')
#         temp['fat'] = soup.find('td', {'itemprop': 'fatContent'}).text
#         temp['name'] = soup.find('td', {'itemprop': 'name'}).text
#         temp['restaurant'] = restaurants[i]['name']
#     print(temp)

In [8]:
# foods = []
# for i in restaurants:
#     url = 'http://chubbygrub.com/restaurants/' + restaurants[i]['slug']
#     response = requests.get(url)
#     soup = BeautifulSoup(response.text, 'lxml')
#     temp = {}
#     temp['calories'] = [i.text for i in soup.find_all('td', {'itemprop': 'calories'})]
#     temp['carbs'] = [i.text for i in soup.find_all('td', {'itemprop': 'carbohydrateContent'})]
#     temp['cateogry'] = [i.text.replace('\n', '') for i in soup.find_all('td', {'class': 'hidden-xs category-column'})]
#     temp['fat'] = [i.text for i in soup.find_all('td', {'itemprop': 'fatContent'})]   
#     temp['name'] = [i.text for i in soup.find_all('td', {'itemprop': 'name'})]
#     foods.append(temp)

In [9]:
calories = []
carbs = []
category = []
fat = []
name = []
restaurant = []
for url in urls:
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'lxml')
    
    for entry in soup.find_all('td', {'itemprop': 'calories'}):
        calories.append(entry.text)
    # calories = [i.text for i in soup.find_all('td', {'itemprop': 'calories'})]
    
    for entry in soup.find_all('td', {'itemprop': 'carbohydrateContent'}):
        carbs.append(entry.text)
        
    for entry in soup.find_all('td', {'class': 'hidden-xs category-column'}):
        category.append(entry.text.replace('\n', ''))
        
    for entry in soup.find_all('td', {'itemprop': 'fatContent'}):
        fat.append(entry.text)
        
    for entry in soup.find_all('td', {'itemprop': 'name'}):
        name.append(entry.text)
    
        for entry in soup.find_all('title'):
            restaurant.append(entry.text.replace(' Nutrition | Calories and Carbs from Your Favorite Restaurants | Chubbygrub', ''))

In [10]:
foods = []
for i in range(len(calories)):
    new_dict = {'calories': calories[i],
        'carbs': carbs[i],
        'category': category[i],
        'fat': fat[i],
        'name': name[i],
        'restaurant': restaurant[i]}
    foods.append(new_dict)
foods[-3:]

[{'calories': '220',
  'carbs': '28',
  'category': 'French Fries',
  'fat': '11',
  'name': 'Value French Fries',
  'restaurant': "Wendy's"},
 {'calories': '220',
  'carbs': '28',
  'category': 'French Fries',
  'fat': '11',
  'name': 'Value French Fries',
  'restaurant': "Wendy's"},
 {'calories': '290',
  'carbs': '53',
  'category': 'Shakes',
  'fat': '6',
  'name': 'Vanilla Frosty Waffle Cone',
  'restaurant': "Wendy's"}]

### Step 4: Create a pandas DataFrame from your list of foods

**Note**: Your DataFrame should have 4,977 rows

In [11]:
# temp = {}
# temp['calories'] = calories
# temp['carbs'] = carbs
# temp['category'] = category
# temp['fat'] = fat
# temp['name'] = name
# temp['restaurant'] = restaurant
# df = pd.DataFrame(temp)
# df.head()

In [12]:
df_alt = pd.DataFrame(foods)
df_alt.head()

,calories,carbs,category,fat,name,restaurant
0,0,0,Drinks,0,A&W® Diet Root Beer,A&W Restaurants
1,0,0,Drinks,0,A&W® Diet Root Beer,A&W Restaurants
2,0,0,Drinks,0,A&W® Diet Root Beer,A&W Restaurants
3,0,0,Drinks,0,A&W® Diet Root Beer,A&W Restaurants
4,0,0,Drinks,0,A&W® Diet Root Beer,A&W Restaurants


In [13]:
df_alt.shape

(4977, 6)

### Step 5: Export to csv

**Note:** Don't export the index column from your DataFrame

In [14]:
df_alt.to_csv('./df.csv', index=False)